In [29]:
import os
import imagesize

import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.notebook import tqdm

In [30]:
resnet = tf.keras.applications.resnet.ResNet152(
    include_top=False
)

171446536/171446536 [==============================] - 11s 0us/step


In [31]:
def extract_features(path):
    image = cv2.imread(path, 0)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    image = cv2.resize(image, (224, 224)).reshape(-1, 224, 224, 3)

    image = tf.keras.applications.resnet.preprocess_input(image)

    features = resnet.predict(image)[0]
    maxs = np.max(features, axis=(0, 1))
    # return  features.reshape(100352, order='F'), maxs
    return  maxs

In [5]:
folders = [
    ('./data/test/reshaped', './data/test/features.csv', './data/test/maxes.csv'),
    ('./data/validation/reshaped', './data/validation/features.csv', './data/validation/maxes.csv'),
    ('./data/train/reshaped', './data/train/features.csv', './data/train/maxes.csv'),
]

# features = []
maxes = []

for folder, feat_path, max_path in folders:
    images = os.listdir(folder)
    for image in tqdm(images):
        path = os.path.join(folder, image)
        image_maxes = extract_features(path)
        # features.append(image_features)
        maxes.append(image_maxes)
    # i_df = pd.DataFrame(features)
    # i_df.insert(0, 'image', images)
    # i_df.to_csv(feat_path, index=False)

    m_df = pd.DataFrame(maxes)
    m_df.insert(0, 'image', images)
    m_df.to_csv(max_path, index=False)

    maxes = []
    # features = []

  0%|          | 0/200 [00:00<?, ?it/s]

1/1 [==============================] - 0s 31ms/step


  0%|          | 0/1424 [00:00<?, ?it/s]

1/1 [==============================] - 0s 32ms/step


  0%|          | 0/12600 [00:00<?, ?it/s]

1/1 [==============================] - 0s 37ms/step


In [6]:
!pip install imagesize

In [0]:
images = os.listdir('./data/train/images')

sizes = (imagesize.get(f'./data/train/images/{image}') for image in images)
longs, shorts = list(zip(*((max(size), min(size)) for size in sizes)))
longs = np.array(longs)
shorts = np.array(shorts)
areas = longs * shorts

In [26]:
assert(all(longs >= shorts))

In [22]:
print(f'Long: {min(longs)} - {np.mean(longs)}  - {np.median(longs)} - {max(longs)}')
print(f'Short: {min(shorts)} - {np.mean(shorts)}  - {np.median(shorts)} - {max(shorts)}')
print(f'Pixels: {min(areas)} - {np.mean(areas)}  - {np.median(areas)} - {max(areas)}')

Long: 677 - 1668.3890254539688  - 1627.0 - 3001
Short: 521 - 1315.2073586551423  - 1275.0 - 2929
Pixels: 387103 - 2292873.6443581  - 2089152.0 - 8765921


In [37]:
np.sqrt(0.0083) * 200

18.2208671582886